In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:

from ctrl.control_system import DiscreteControlSystem
import matplotlib.pyplot as plt
from copy import copy
import numpy as np


def model(x, u):
    F = 0.0015
    G = 0.0025
    x_next = copy(x)
    x_next[0] += x[1]
    x_next[1] += u * F - np.cos(3 * x[0]) * G

    return x_next


# define plant
dt = 1

plant_output = lambda x, u: x

plant = DiscreteControlSystem(model, plant_output, dim_x=2, dim_u=1, dim_y=2, dt=dt)

In [ ]:
from ctrl.controllers.lqr import LQRController
from ctrl.controllers.random import RandomController
from ctrl.control_system import linearize_system

# define cost function
cost = lambda x, u : (x[0] - 0.5)**2 + u**2

# create a dictionary of controllers

# 1. LQR around (-0.52, 0) point
Q = np.eye(2)
R = np.eye(1)
x0 = [(np.arccos(0) - np.pi)/ 3, 0]
u0 = 0
lin_plant = linearize_system(plant, x0, u0)
lqr = LQRController(lin_plant, Q, R, x_ref=x0, u_ref=u0)

# 2. LQR around (0.52, 0) point
x0 = [np.arccos(0)/ 3, 0]
u0 = 0
lin_plant_2 = linearize_system(plant, x0, u0)
lqr_2 = LQRController(lin_plant, Q, R, x_ref=x0, u_ref=u0)

# 3. Random controller
rnd_ctrl = RandomController(-1, 1)

# 4. Bang-bang
from ctrl.controllers.controller import StateFeedbackController

class BangBang(StateFeedbackController):
    def compute_control(self, x, t):
        if x[0] < 0.5:
            return 1.0
        else:
            return -1.0

class ReverseBangBang(StateFeedbackController):
    def compute_control(self, x, t):
        if x[0] > 0.5:
            return 1.0
        else:
            return -1.0

bb_ctrl = BangBang()
revbb_ctrl = ReverseBangBang()

controllers = [lqr, lqr_2, rnd_ctrl, bb_ctrl]
controllers = [lqr, lqr_2, rnd_ctrl, bb_ctrl, revbb_ctrl]
controllers = [lqr_2]

In [ ]:
from ctrl.controllers.mcts import MCTSController

# create MCTS
x0 = [-0.5, 0]
x_ref = 0.5
cost_function = lambda x, u: -np.sum((x[0] - x_ref) ** 2 + 0.1 * u ** 2)
mcts = MCTSController(tree_size=100, sim_time=20, control_system=plant,
                      controllers=controllers, base_controller=bb_ctrl,
                      cost_function=cost_function)



In [ ]:
from ctrl.simulation import DiscreteControllerPlantObserverLoop

sim = DiscreteControllerPlantObserverLoop(mcts, plant)

x, _, _, u = sim.sim(x0=x0, t=200)

import matplotlib.pyplot as plt
plt.plot(x)
plt.plot(u)
plt.legend(['x', 'v', 'u'])

In [ ]:
# import gymnasium as gym
# env = gym.make('MountainCarContinuous-v0', render_mode="human")
# observation, info = env.reset()
#
# for _ in range(1000):
#     action = [bb_ctrl(list(observation))]
#     observation, reward, terminated, truncated, info = env.step(action)
#
#     if terminated or truncated:
#         env.close()

In [22]:
lqr_2(np.array([0, 0]))
x0 = (np.arccos(0) - np.pi)/ 3
model([x0, 0], 0)

[-0.5235987755982988, -1.5308084989341915e-19]